In [1]:
import pandas as pd
import time
from tqdm._tqdm_notebook import tqdm_notebook
import numpy as np
tqdm_notebook.pandas()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id='***REMOVED***',
                                                      client_secret='***REMOVED***')
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [173]:
query = spotify.search('Quiero Club	Let Da Music', type='track', market='US')
# if query['tracks']['items'][0]['id'][1:] == song_info.iloc[0]['song_id']:
#     print("Match!")
for item in query['tracks']['items']:
    print(item['id'])

7pfDPQjEnsaaNxfMd4Kn3X


In [170]:
available_markets = ["AD","AR","AT","AU","BE","BG","BO","BR","CA","CH","CL","CO","CR","CY","CZ",
      "DE","DK","DO","EC","EE","ES","FI","FR","GB","GR","GT","HK","HN","HU","ID","IE","IL","IS",
      "IT","JP","LI","LT","LU","LV","MC","MT","MX","MY","NI","NL","NO","NZ","PA","PE","PH","PL",
      "PT","PY","RO","SE","SG","SK","SV","TH","TR","TW","US","UY","VN","ZA"]


def fix_ids(row):
    time.sleep(.01)
    if row['location'][-2:] in available_markets:
        try:
            query = spotify.search(q=row['artist'] + ' ' + row['title'], market=row['location'][-2:], type='track')
            for item in query['tracks']['items']:
                if item['id'][1:] == row['song_id']:
                    return item['id']
        except Exception:
            pass
    
    query = spotify.search(q=row['artist'] + ' ' + row['title'], market='US', type='track')
    for item in query['tracks']['items']:
            if item['id'][1:] == row['song_id']:
                return item['id']
            
    return np.nan


def fix_ids_brute(row):
    time.sleep(.5)
    for c in '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ':
        try:
            result = spotify.track(c+row['song_id'])
#             if result['name'] == row['title']:
            return result['id']
        except:
            pass
    return np.nan
# spotify:track:6FLSb8CJwrcfXfzfD4ySji
# lyrics = pd.read_csv('data/lyrics_with_lang.txt')
# # lyrics=lyrics[16448:16500]
# lyrics['id_fix'] = lyrics.progress_apply(lambda row:fix_ids(row), axis=1)
# lyrics

### Generate IDs that need fixing

In [253]:
file_name = 'songs_2019-3-1_11-31-2.txt'
file_path = 'data/songs/bad_ids/'+file_name
output_file_path = 'data/songs/'+file_name

In [259]:
songs = pd.read_csv(file_path)
id_map = pd.read_csv('data/id_map.txt')
songs_fix = songs.merge(right=id_map, how='left', on='song_id')
songs_fix.head()

,location,playlist_id,artist,title,song_id,id_fix
0,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Alu Mix,Le Encanta,cX3Om7xlCQxKRYo1ofvOM,2cX3Om7xlCQxKRYo1ofvOM
1,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Uzielito Mix,Perreo Que Te Vuelve Loca,8eFISo4m37yTdLeJzVdqn,08eFISo4m37yTdLeJzVdqn
2,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Cheo Andujar,Dia Tras Dia,EJadbufi05YD8vQb98Poq,6EJadbufi05YD8vQb98Poq
3,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Michael G,Tibiri Tabara - feat. Uzielito Mix,Ulyr7Y5DFPnH7Qo3qxvFL,2Ulyr7Y5DFPnH7Qo3qxvFL
4,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Sir Speedy,Como Mi Nena No Hay (feat. Master Joe y O.G. B...,33FRL9y0I7Pm1QZBeMm6X,733FRL9y0I7Pm1QZBeMm6X


location                 Khon Kaen TH
playlist_id    4Q8IBRgy9EwSPWZwoJWDOp
artist                             KS
title                             ยืด
song_id         hSAMX0YTjCOeGrZcN1za2
Name: 201836, dtype: object

In [260]:
print(len(songs_fix))

id_needed = songs_fix[songs_fix['id_fix'].isnull()].copy()
print(len(id_needed))
id_needed.drop_duplicates(subset=['song_id'], inplace=True)
print(len(id_needed))

225665
0
0


### Use one of two methods to regenerate Spotify ID

In [258]:

# id_needed['id_fix'] = id_needed.progress_apply(lambda row:fix_ids(row), axis=1)

# id_needed['id_fix'] = id_needed.progress_apply(lambda row:fix_ids_brute(row), axis=1)
id_needed

,location,playlist_id,artist,title,song_id,id_fix
202543,Thanyaburi TH,4LEEZN3eNXVlehjvEx3yKL,"KS"",ยืด""",hSAMX0YTjCOeGrZcN1za2,NaN,NaN


In [240]:
print(len(id_needed))
# id_needed.to_csv(path_or_buf='./data/id_needed.txt', index=False, encoding='utf-8')
still_missing = id_needed[id_needed['id_fix'].isnull()].copy()
print(len(still_missing))

421
1


In [241]:
id_needed.dropna(inplace=True, subset=['id_fix'])

In [242]:
id_map = pd.read_csv('data/id_map.txt')
print(len(id_map))
id_map = id_map.append(id_needed[['song_id', 'id_fix']], ignore_index=True)
print(len(id_map))

65262
65682


In [243]:
id_map.drop_duplicates(subset=['id_fix'], inplace=True)
id_map.dropna(subset=['id_fix'], inplace=True)
print(len(id_map))


65681


In [244]:
id_map.to_csv(path_or_buf='./data/id_map.txt', index=False, encoding='utf-8')

In [249]:
still_missing
for c in '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ':
        try:
            result = spotify.track(c+'hSAMX0YTjCOeGrZcN1za2')
#             if result['name'] == row['title']:

        except:
            pass
        
result

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/58nieOasZGIRwwXt2Jp7Pf'},
    'href': 'https://api.spotify.com/v1/artists/58nieOasZGIRwwXt2Jp7Pf',
    'id': '58nieOasZGIRwwXt2Jp7Pf',
    'name': 'KS"',
    'type': 'artist',
    'uri': 'spotify:artist:58nieOasZGIRwwXt2Jp7Pf'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/63WXulpO6XrHyU49lp3NLp'},
    'href': 'https://api.spotify.com/v1/artists/63WXulpO6XrHyU49lp3NLp',
    'id': '63WXulpO6XrHyU49lp3NLp',
    'name': 'FOMEO',
    'type': 'artist',
    'uri': 'spotify:artist:63WXulpO6XrHyU49lp3NLp'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS'

In [236]:
# songs.loc['201836']['title'] = 'ยืด'
# songs.loc[201836]['title'] = 'ยืด'
# songs.loc[201836]['song_id'] = 'hSAMX0YTjCOeGrZcN1za2'
songs.loc[201836]
# result
# songs.to_csv(file_path, index=False)

location                 Khon Kaen TH
playlist_id    4Q8IBRgy9EwSPWZwoJWDOp
artist                       KS",ยืด"
title           hSAMX0YTjCOeGrZcN1za2
song_id                           NaN
Name: 201836, dtype: object

### Merge fixed IDs from new ID Map into the songs file

In [261]:
songs = pd.read_csv(file_path)
id_map = pd.read_csv('data/id_map.txt')

In [262]:
songs = songs.merge(right=id_map, how='left', on='song_id')
assert(len(songs[songs['id_fix'].isnull()]) == 0)
songs.head()

,location,playlist_id,artist,title,song_id,id_fix
0,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Alu Mix,Le Encanta,cX3Om7xlCQxKRYo1ofvOM,2cX3Om7xlCQxKRYo1ofvOM
1,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Uzielito Mix,Perreo Que Te Vuelve Loca,8eFISo4m37yTdLeJzVdqn,08eFISo4m37yTdLeJzVdqn
2,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Cheo Andujar,Dia Tras Dia,EJadbufi05YD8vQb98Poq,6EJadbufi05YD8vQb98Poq
3,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Michael G,Tibiri Tabara - feat. Uzielito Mix,Ulyr7Y5DFPnH7Qo3qxvFL,2Ulyr7Y5DFPnH7Qo3qxvFL
4,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Sir Speedy,Como Mi Nena No Hay (feat. Master Joe y O.G. B...,33FRL9y0I7Pm1QZBeMm6X,733FRL9y0I7Pm1QZBeMm6X


In [263]:
songs.drop(columns=['song_id'], inplace=True)
songs.rename(columns={'id_fix': 'song_id'}, inplace=True)
songs.head()

,location,playlist_id,artist,title,song_id
0,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Alu Mix,Le Encanta,2cX3Om7xlCQxKRYo1ofvOM
1,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Uzielito Mix,Perreo Que Te Vuelve Loca,08eFISo4m37yTdLeJzVdqn
2,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Cheo Andujar,Dia Tras Dia,6EJadbufi05YD8vQb98Poq
3,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Michael G,Tibiri Tabara - feat. Uzielito Mix,2Ulyr7Y5DFPnH7Qo3qxvFL
4,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Sir Speedy,Como Mi Nena No Hay (feat. Master Joe y O.G. B...,733FRL9y0I7Pm1QZBeMm6X


In [264]:
songs.to_csv(output_file_path, index=False)

### Replace Bad IDs

In [38]:
input_file_path = 'data/bad_ids/lyrics_with_lang.txt'
output_file_path = 'data/lyrics_with_lang.txt'

bad_ids = pd.read_csv(input_file_path)
id_map = pd.read_csv('data/id_map.txt') 
len(id_map)

55216

In [39]:
fixed = bad_ids.merge(right=id_map, how='left', on='song_id')
fixed

,location,artist,title,song_id,lyrics,lang,id_fix
0,Mexico City MX,Grupo Polo Montañes,Un Sueño Y Nada Mas,bVisOYbfWI29XxWEdTKSX,A veces tengo ganas de volver a verte\r\nY sé ...,es,2bVisOYbfWI29XxWEdTKSX
1,Mexico City MX,Los Estramboticos,La Herida,vQfzyjQFHQ208JCNjcE5H,Fue esta soledad y no el destino\r\nQuien nos ...,es,0vQfzyjQFHQ208JCNjcE5H
2,Mexico City MX,La Castañeda,Cenit,hLo5Dg74PGHjSbcP6tNOr,Creí cuando dijeron que la luna se dormía cuan...,es,6hLo5Dg74PGHjSbcP6tNOr
3,Mexico City MX,Maskatesta,Tu Que Pediras,84Ztj2XGMvoWKILVJGasd,"Tee diiree una vez mass, lo quee pido por tii\...",es,284Ztj2XGMvoWKILVJGasd
4,Mexico City MX,DLD,Por Siempre,FLSb8CJwrcfXfzfD4ySji,Hoy he vuelto a ser quien soy\r\nla vida me re...,es,6FLSb8CJwrcfXfzfD4ySji
5,Mexico City MX,Los De Abajo,Skapate,mGNPivC5BpTowR3NF45bD,Uhh mami\r\nCon este ritmo va que me sabe a ch...,es,0mGNPivC5BpTowR3NF45bD
6,Mexico City MX,Los Caligaris,Kilómetros,Jr2cHZaIe1rmdVlc5efIy,Cada vez que pienso y me doy cuenta donde esto...,es,4Jr2cHZaIe1rmdVlc5efIy
7,Mexico City MX,Maria Daniela Y Su Sonido Lasser,Pobre Estúpida,Vu0aOYDeVPLe4dcVrvRvx,"Y ella de qué va, siempre está mirando atrás.\...",es,0Vu0aOYDeVPLe4dcVrvRvx
8,Mexico City MX,La Gusana Ciega,"Yes Sir, I Can Boogie",IwpFZxC7TlYBwVpXqks8n,Oye \r\nEstas perdiendo mucho tiempo \r\nNo du...,es,5IwpFZxC7TlYBwVpXqks8n
9,Mexico City MX,Liquits,Desde Que,QLfVqOrpBOJra53EhlEX0,"Te senti venir como en una vision, \r\nYa esta...",es,4QLfVqOrpBOJra53EhlEX0


In [40]:
fixed.drop(columns=['song_id'], inplace=True)
fixed.rename(columns={'id_fix': 'song_id'}, inplace=True)
fixed.head()

,location,artist,title,lyrics,lang,song_id
0,Mexico City MX,Grupo Polo Montañes,Un Sueño Y Nada Mas,A veces tengo ganas de volver a verte\r\nY sé ...,es,2bVisOYbfWI29XxWEdTKSX
1,Mexico City MX,Los Estramboticos,La Herida,Fue esta soledad y no el destino\r\nQuien nos ...,es,0vQfzyjQFHQ208JCNjcE5H
2,Mexico City MX,La Castañeda,Cenit,Creí cuando dijeron que la luna se dormía cuan...,es,6hLo5Dg74PGHjSbcP6tNOr
3,Mexico City MX,Maskatesta,Tu Que Pediras,"Tee diiree una vez mass, lo quee pido por tii\...",es,284Ztj2XGMvoWKILVJGasd
4,Mexico City MX,DLD,Por Siempre,Hoy he vuelto a ser quien soy\r\nla vida me re...,es,6FLSb8CJwrcfXfzfD4ySji


In [41]:
print(len(fixed))
fixed.dropna(subset=['song_id'], inplace=True)
print(len(fixed))

25325
24331


In [42]:
fixed.to_csv(path_or_buf=output_file_path, index=False, encoding='utf-8')

In [5]:
bad_ids = pd.read_csv('./data/songs/bad_ids/songs_2019-3-17_12-31-9.txt')
bad_ids.drop_duplicates(subset=['song_id'], inplace=True)
id_map = pd.read_csv('./data/id_map.txt')

In [40]:
missing = bad_ids.merge(id_map, how='left', on='song_id')
missing = missing[missing['id_fix'].isnull()].copy()
len(missing)


1372

In [41]:
missing

,location,playlist_id,artist,title,song_id,id_fix
1,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Uzielito Mix,Niña Buena,3lqNStHY7nYTCp5qOjppu,NaN
79,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Nortec Collective Presents: Bostich+Fussible,Tijuana Sound Machine,QD7y440mt5uvoDeMPHQai,NaN
88,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Plastilina Mosh,Pervert Pop Song,dkGPPewLZjer3U2pQGIOK,NaN
218,São Paulo BR,6QOjOvLDWKVNCW9H1J9vCY,Consciência Humana,Lembranças,yViudNjrI9luiW3BVmb7z,NaN
231,São Paulo BR,6QOjOvLDWKVNCW9H1J9vCY,Turma do Pagode,"A Batucada dos Nossos Tantãns / Vai Lá, Vai Lá...",9stztMkuaAHcbkuPrElnb,NaN
308,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,1takejay,Hello,auP0o6TOXgzZEJM46yJ9j,NaN
329,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,The Elgins,Darling Baby - Mono,6EcsKLusAz7ydla0ENGkg,NaN
368,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,Tony Allen,Nite Owl,FsEGMk5pEtrChLU8etKBZ,NaN
390,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,Azjah,Time for It,XbHTAmnCUpm24jCaXoYRD,NaN
392,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,Shordie Shordie,Bitchuary,mpYhrHLVOqG0AnXVPf1Xq,NaN


In [42]:
query = spotify.search('Plastilina Mosh	Pervert Pop Song', type='track', market='MX')
# if query['tracks']['items'][0]['id'][1:] == song_info.iloc[0]['song_id']:
#     print("Match!")
for item in query['tracks']['items']:
    print(item)
#     print(item['id'])

{'album': {'album_type': 'compilation', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'}, 'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of', 'id': '0LyfQWJT6nXafLPZqxe9Of', 'name': 'Various Artists', 'type': 'artist', 'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/6NS5jRu0SnahlW9StQqoNO'}, 'href': 'https://api.spotify.com/v1/albums/6NS5jRu0SnahlW9StQqoNO', 'id': '6NS5jRu0SnahlW9StQqoNO', 'images': [{'height': 600, 'url': 'https://i.scdn.co/image/3e9d8b9a89a1a7651e1f036c47d726e7a46dadd0', 'width': 600}, {'height': 300, 'url': 'https://i.scdn.co/image/2e2268ad54db2c7c9a6c1152679db77d3c5f6c71', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/4955a04a640e51b2c8ca1a22ed99d62c8fc67813', 'width': 64}], 'name': 'Pop...¡Por La Cara!', 'release_date': '2018-08-17', 'release_date_precision': 'day', 'total_tracks': 29, 'type': 'album', 'uri': 'spotify:a

In [44]:
# missing = missing[:10]
missing['id_fix'] = missing.progress_apply(lambda row:fix_ids_brute(row), axis=1)
missing

,location,playlist_id,artist,title,song_id,id_fix
1,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Uzielito Mix,Niña Buena,3lqNStHY7nYTCp5qOjppu,23lqNStHY7nYTCp5qOjppu
79,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Nortec Collective Presents: Bostich+Fussible,Tijuana Sound Machine,QD7y440mt5uvoDeMPHQai,2QD7y440mt5uvoDeMPHQai
88,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Plastilina Mosh,Pervert Pop Song,dkGPPewLZjer3U2pQGIOK,1dkGPPewLZjer3U2pQGIOK
218,São Paulo BR,6QOjOvLDWKVNCW9H1J9vCY,Consciência Humana,Lembranças,yViudNjrI9luiW3BVmb7z,0yViudNjrI9luiW3BVmb7z
231,São Paulo BR,6QOjOvLDWKVNCW9H1J9vCY,Turma do Pagode,"A Batucada dos Nossos Tantãns / Vai Lá, Vai Lá...",9stztMkuaAHcbkuPrElnb,69stztMkuaAHcbkuPrElnb
308,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,1takejay,Hello,auP0o6TOXgzZEJM46yJ9j,4auP0o6TOXgzZEJM46yJ9j
329,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,The Elgins,Darling Baby - Mono,6EcsKLusAz7ydla0ENGkg,66EcsKLusAz7ydla0ENGkg
368,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,Tony Allen,Nite Owl,FsEGMk5pEtrChLU8etKBZ,7FsEGMk5pEtrChLU8etKBZ
390,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,Azjah,Time for It,XbHTAmnCUpm24jCaXoYRD,2XbHTAmnCUpm24jCaXoYRD
392,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,Shordie Shordie,Bitchuary,mpYhrHLVOqG0AnXVPf1Xq,7mpYhrHLVOqG0AnXVPf1Xq


In [122]:
missing[['song_id', 'id_fix']].to_csv('missing_map.txt', index=False)

In [46]:
print(len(missing.dropna(subset=['id_fix'])))
len(missing)

1344


1372

In [64]:
# still_missing = missing[missing['id_fix'].isnull()].copy()
still_missing['id_fix'] = still_missing.progress_apply(lambda row:fix_ids_brute(row), axis=1)
still_missing

,location,playlist_id,artist,title,song_id,id_fix
2223,Zapopan MX,3L11cPKOzKS0bwlo8ljj8Z,Julion A.,Ojos Verdes,kwgIiCzl9fDZE0so4bDbS,4kwgIiCzl9fDZE0so4bDbS
3792,Guadalajara MX,75kkQ1mtJJBqHFWaJyxip8,Julion A.,Cuando Te Ame,sRqLvNGFUgMNLp4d1usBt,3sRqLvNGFUgMNLp4d1usBt
5157,Cologne DE,1qvxfB4gqksSukajXmsM6x,Telson,Um mich rum (feat. Sukh Andalous & Shaan A.),8ea6Tep0qSMpzTlZ7L01v,48ea6Tep0qSMpzTlZ7L01v
6228,Frankfurt am Main DE,4jloudldUXdq0anSBoKoZw,Berkan,Kettensäge (feat. Olexesh),MilBAn0jRjybQPbY2p7nY,0MilBAn0jRjybQPbY2p7nY
6789,Zurich CH,4ywSNVTg3bsD9PbfnVtxa1,Jakko Bratko,Ade Madame (feat. Chekaa),u86rBHx946WkiEdEbNffs,6u86rBHx946WkiEdEbNffs
14654,Nottingham GB,4Brr9Wqz3bDD4UiBpeX18h,The Enemy UK,Away From Here - US Version,K5yg8zFm5gJb1focIZZMQ,0K5yg8zFm5gJb1focIZZMQ
15442,Chihuahua City MX,77WXt3OdU7OeyI5x8lx3Vc,Julion A.,Soñe,kV2h281bcXw4IJSy1PXks,2kV2h281bcXw4IJSy1PXks
16913,Morelia MX,7zv9G9oKR1hUrZ8ZaTxHSC,Julion A.,Como Este Cabron,B5PDXqVjvCngvCQLtr2HQ,2B5PDXqVjvCngvCQLtr2HQ
17439,Buffalo New York US,4StWFoCs3vp4gDxAGCwybU,Can't Swim,sometimes you meet the right people at the wro...,rb3g5SKaWMsV6v7mI9GQQ,0rb3g5SKaWMsV6v7mI9GQQ
18873,Villahermosa MX,0erw4Jyb5cNakOA6uWX4NM,Julion A.,La Sonrisa Obligatoria,IfYatzvdtsB2wYvbT24AM,2IfYatzvdtsB2wYvbT24AM


In [67]:
still_missing.to_csv('still_missing.csv')

In [65]:
new_id_map = id_map.append(still_missing[['song_id', 'id_fix']], ignore_index=True)

In [108]:
len(still_missing)

28

In [51]:
missing[-1:]

,location,playlist_id,artist,title,song_id,id_fix
48878,Erp NL,3fUwdKgKLkLNXorGJLYuf9,Ismo,Aan De Kant,C0e3u2KuB8i6nXqWP4elk,7C0e3u2KuB8i6nXqWP4elk


In [77]:
id_map.to_csv(path_or_buf='./data/id_map.txt', index=False, encoding='utf-8')

In [63]:
print(fix_ids_brute(still_missing.iloc[4]))

6u86rBHx946WkiEdEbNffs


In [73]:
for i, row in still_missing.iterrows():
    print(spotify.track(row['id_fix'])['artists'][0]['name'])



Telson
Berkan
Jakko Bratko
The Enemy


Can't Swim

Happy Mondays
Labi Siffre
Ligabue
Fauve
Anji
T3R Elemento

Sandy & Junior
Alan Menken
Dario Marianelli
Sudor Marika
Wiktoria Johansson
Udit Narayan
Allj

Dylisa
Vijay
Afro B


In [100]:
new_map = pd.read_csv('./data/id_map.txt')

In [101]:
len(new_map)

55216

In [76]:
len(bad_ids)

48921

In [79]:
bad_ids = pd.read_csv('./data/songs/bad_ids/songs_2019-3-17_12-31-9.txt')
a = bad_ids.merge(id_map, how='left', on='song_id')
a = a[a['id_fix'].isnull()].copy()
len(a)

5026

In [102]:
new_map = new_map.append(missing[['song_id', 'id_fix']])

In [119]:
len(new_map)
still_missing[['song_id', 'id_fix']].to_csv('still_missing_map.txt', index=False)

In [106]:
new_map = new_map.append(still_missing[['song_id', 'id_fix']], ignore_index=True)

In [89]:
a = bad_ids.merge(id_map, how='left', on='song_id')
a = a[a['id_fix'].isnull()].copy()
len(a)

138

In [115]:
still_missing

,location,playlist_id,artist,title,song_id,id_fix
2223,Zapopan MX,3L11cPKOzKS0bwlo8ljj8Z,Julion A.,Ojos Verdes,kwgIiCzl9fDZE0so4bDbS,4kwgIiCzl9fDZE0so4bDbS
3792,Guadalajara MX,75kkQ1mtJJBqHFWaJyxip8,Julion A.,Cuando Te Ame,sRqLvNGFUgMNLp4d1usBt,3sRqLvNGFUgMNLp4d1usBt
5157,Cologne DE,1qvxfB4gqksSukajXmsM6x,Telson,Um mich rum (feat. Sukh Andalous & Shaan A.),8ea6Tep0qSMpzTlZ7L01v,48ea6Tep0qSMpzTlZ7L01v
6228,Frankfurt am Main DE,4jloudldUXdq0anSBoKoZw,Berkan,Kettensäge (feat. Olexesh),MilBAn0jRjybQPbY2p7nY,0MilBAn0jRjybQPbY2p7nY
6789,Zurich CH,4ywSNVTg3bsD9PbfnVtxa1,Jakko Bratko,Ade Madame (feat. Chekaa),u86rBHx946WkiEdEbNffs,6u86rBHx946WkiEdEbNffs
14654,Nottingham GB,4Brr9Wqz3bDD4UiBpeX18h,The Enemy UK,Away From Here - US Version,K5yg8zFm5gJb1focIZZMQ,0K5yg8zFm5gJb1focIZZMQ
15442,Chihuahua City MX,77WXt3OdU7OeyI5x8lx3Vc,Julion A.,Soñe,kV2h281bcXw4IJSy1PXks,2kV2h281bcXw4IJSy1PXks
16913,Morelia MX,7zv9G9oKR1hUrZ8ZaTxHSC,Julion A.,Como Este Cabron,B5PDXqVjvCngvCQLtr2HQ,2B5PDXqVjvCngvCQLtr2HQ
17439,Buffalo New York US,4StWFoCs3vp4gDxAGCwybU,Can't Swim,sometimes you meet the right people at the wro...,rb3g5SKaWMsV6v7mI9GQQ,0rb3g5SKaWMsV6v7mI9GQQ
18873,Villahermosa MX,0erw4Jyb5cNakOA6uWX4NM,Julion A.,La Sonrisa Obligatoria,IfYatzvdtsB2wYvbT24AM,2IfYatzvdtsB2wYvbT24AM


In [128]:
bad_ids = pd.read_csv('./data/songs/bad_ids/songs_2019-3-17_12-31-9.txt')
new_map = pd.read_csv('./data/final_id_map.txt')
a = bad_ids.merge(new_map, how='left', on='song_id')
a = a[a['id_fix'].isnull()].copy()
len(a)

0

In [116]:
a = a.merge(still_missing[['song_id', 'id_fix']], how='left', on='song_id')

In [121]:
a

,location,playlist_id,artist,title,song_id,id_fix
1,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Uzielito Mix,Niña Buena,3lqNStHY7nYTCp5qOjppu,NaN
79,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Nortec Collective Presents: Bostich+Fussible,Tijuana Sound Machine,QD7y440mt5uvoDeMPHQai,NaN
88,Mexico City MX,5SHk94ZLUYckc8ZQBF4vqv,Plastilina Mosh,Pervert Pop Song,dkGPPewLZjer3U2pQGIOK,NaN
218,São Paulo BR,6QOjOvLDWKVNCW9H1J9vCY,Consciência Humana,Lembranças,yViudNjrI9luiW3BVmb7z,NaN
231,São Paulo BR,6QOjOvLDWKVNCW9H1J9vCY,Turma do Pagode,"A Batucada dos Nossos Tantãns / Vai Lá, Vai Lá...",9stztMkuaAHcbkuPrElnb,NaN
308,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,1takejay,Hello,auP0o6TOXgzZEJM46yJ9j,NaN
329,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,The Elgins,Darling Baby - Mono,6EcsKLusAz7ydla0ENGkg,NaN
368,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,Tony Allen,Nite Owl,FsEGMk5pEtrChLU8etKBZ,NaN
390,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,Azjah,Time for It,XbHTAmnCUpm24jCaXoYRD,NaN
392,Los Angeles California US,53JqQphsAHvDGGVZErKmW5,Shordie Shordie,Bitchuary,mpYhrHLVOqG0AnXVPf1Xq,NaN


In [153]:
new_map = pd.read_csv('./data/id_map.txt')
print(len(new_map))
new_map = new_map.append(missing[['song_id', 'id_fix']])
print(len(new_map))
new_map.dropna(subset=['id_fix'], inplace=True)

57044
57500


In [133]:
new_map.to_csv('./data/final_id_map.txt', index=False)

In [129]:
lyrics = pd.read_csv('./data/bad_ids/lyrics_with_lang.txt')

In [130]:
a = lyrics.merge(new_map, how='left', on='song_id')
a = a[a['id_fix'].isnull()].copy()
len(a)

456

In [131]:
a

,location,artist,title,song_id,lyrics,lang,Unnamed: 0,id_fix
101,São Paulo BR,Criolo,Grajauex,D5sRscjhPXgpphwIdrDEP,The Grajauex\r\nDuas laje é triplex\r\nNo morr...,pt,NaN,NaN
182,Los Angeles California US,Material Issue,Valerie Loves Me,tqASnM026iYJFzcb2fwa8,"Valerie's dancing, in the room above my bed yo...",en,NaN,NaN
382,Houston Texas US,Honeybrowne,Texas Angel,wVkovdZKbcgwjwjUvQAAj,Well I'm sittin' here all alone wonderin' how ...,en,NaN,NaN
392,Houston Texas US,Cory Morrow,21 Days,giySMOHPQMb1nD3s5OCTs,"I'm going to Carolina\r\nTaking that hard, rou...",en,NaN,NaN
551,Lima PE,Yahaira Plasencia,Tú,iosPMWqMwHeL6cmE1rrZS,"[Letra de ""Tú""]\r\n\r\nEn mi mente estas\r\nCo...",es,NaN,NaN
582,Atlanta Georgia US,T.I.,Do It,YvPWESHazN6QdbEJMp7F4,You looking good in them guess's shawty showin...,en,NaN,NaN
698,New York New York US,Dirty Projectors,Stillness Is The Move,77XuL6LgQdlwrbuEVClDh,When the child was just a child\r\nIt did not ...,en,NaN,NaN
706,New York New York US,Jenny Lewis,Heads Gonna Roll,j8SIEQwxT7I4o6quMIpaa,Since I haven't talked to you I dream about yo...,en,NaN,NaN
712,New York New York US,Yeasayer,2080,SivPyLsl3MjDrnZouZU7O,I can't sleep when I think about the times we'...,en,NaN,NaN
714,New York New York US,Sam Martin,Sugar Is Sweet,6xbrme2m7gV1qx23PsSnM,[Intro]\r\n(Those summer days are forever\r\nT...,en,NaN,NaN


In [138]:
a.drop(columns=['Unnamed: 0'], inplace=True)

In [142]:
missing

,location,artist,title,song_id,lyrics,lang,id_fix
101,São Paulo BR,Criolo,Grajauex,D5sRscjhPXgpphwIdrDEP,The Grajauex\r\nDuas laje é triplex\r\nNo morr...,pt,NaN
182,Los Angeles California US,Material Issue,Valerie Loves Me,tqASnM026iYJFzcb2fwa8,"Valerie's dancing, in the room above my bed yo...",en,NaN
382,Houston Texas US,Honeybrowne,Texas Angel,wVkovdZKbcgwjwjUvQAAj,Well I'm sittin' here all alone wonderin' how ...,en,NaN
392,Houston Texas US,Cory Morrow,21 Days,giySMOHPQMb1nD3s5OCTs,"I'm going to Carolina\r\nTaking that hard, rou...",en,NaN
551,Lima PE,Yahaira Plasencia,Tú,iosPMWqMwHeL6cmE1rrZS,"[Letra de ""Tú""]\r\n\r\nEn mi mente estas\r\nCo...",es,NaN
582,Atlanta Georgia US,T.I.,Do It,YvPWESHazN6QdbEJMp7F4,You looking good in them guess's shawty showin...,en,NaN
698,New York New York US,Dirty Projectors,Stillness Is The Move,77XuL6LgQdlwrbuEVClDh,When the child was just a child\r\nIt did not ...,en,NaN
706,New York New York US,Jenny Lewis,Heads Gonna Roll,j8SIEQwxT7I4o6quMIpaa,Since I haven't talked to you I dream about yo...,en,NaN
712,New York New York US,Yeasayer,2080,SivPyLsl3MjDrnZouZU7O,I can't sleep when I think about the times we'...,en,NaN
714,New York New York US,Sam Martin,Sugar Is Sweet,6xbrme2m7gV1qx23PsSnM,[Intro]\r\n(Those summer days are forever\r\nT...,en,NaN


In [143]:
missing['id_fix'] = missing.progress_apply(lambda row:fix_ids_brute(row), axis=1)
missing

,location,artist,title,song_id,lyrics,lang,id_fix
101,São Paulo BR,Criolo,Grajauex,D5sRscjhPXgpphwIdrDEP,The Grajauex\r\nDuas laje é triplex\r\nNo morr...,pt,6D5sRscjhPXgpphwIdrDEP
182,Los Angeles California US,Material Issue,Valerie Loves Me,tqASnM026iYJFzcb2fwa8,"Valerie's dancing, in the room above my bed yo...",en,0tqASnM026iYJFzcb2fwa8
382,Houston Texas US,Honeybrowne,Texas Angel,wVkovdZKbcgwjwjUvQAAj,Well I'm sittin' here all alone wonderin' how ...,en,0wVkovdZKbcgwjwjUvQAAj
392,Houston Texas US,Cory Morrow,21 Days,giySMOHPQMb1nD3s5OCTs,"I'm going to Carolina\r\nTaking that hard, rou...",en,1giySMOHPQMb1nD3s5OCTs
551,Lima PE,Yahaira Plasencia,Tú,iosPMWqMwHeL6cmE1rrZS,"[Letra de ""Tú""]\r\n\r\nEn mi mente estas\r\nCo...",es,3iosPMWqMwHeL6cmE1rrZS
582,Atlanta Georgia US,T.I.,Do It,YvPWESHazN6QdbEJMp7F4,You looking good in them guess's shawty showin...,en,5YvPWESHazN6QdbEJMp7F4
698,New York New York US,Dirty Projectors,Stillness Is The Move,77XuL6LgQdlwrbuEVClDh,When the child was just a child\r\nIt did not ...,en,477XuL6LgQdlwrbuEVClDh
706,New York New York US,Jenny Lewis,Heads Gonna Roll,j8SIEQwxT7I4o6quMIpaa,Since I haven't talked to you I dream about yo...,en,4j8SIEQwxT7I4o6quMIpaa
712,New York New York US,Yeasayer,2080,SivPyLsl3MjDrnZouZU7O,I can't sleep when I think about the times we'...,en,6SivPyLsl3MjDrnZouZU7O
714,New York New York US,Sam Martin,Sugar Is Sweet,6xbrme2m7gV1qx23PsSnM,[Intro]\r\n(Those summer days are forever\r\nT...,en,56xbrme2m7gV1qx23PsSnM


In [147]:
test = missing.dropna(subset=['id_fix'])
print(len(test))
# len(missing)

456


In [149]:
missing = missing[['song_id', 'id_fix']]

In [152]:
missing.to_csv('./data/missing2.txt', index=False)

In [158]:
new_map = pd.read_csv('./data/id_map.txt')
lyrics = pd.read_csv('./data/bad_ids/lyrics_with_lang.txt')
lyrics = lyrics.merge(new_map, how='left', on='song_id')
# a = a[a['id_fix'].isnull()].copy()
# len(a)

In [159]:
lyrics

,location,artist,title,song_id,lyrics,lang,id_fix
0,Mexico City MX,Grupo Polo Montañes,Un Sueño Y Nada Mas,bVisOYbfWI29XxWEdTKSX,A veces tengo ganas de volver a verte\r\nY sé ...,es,2bVisOYbfWI29XxWEdTKSX
1,Mexico City MX,Los Estramboticos,La Herida,vQfzyjQFHQ208JCNjcE5H,Fue esta soledad y no el destino\r\nQuien nos ...,es,0vQfzyjQFHQ208JCNjcE5H
2,Mexico City MX,La Castañeda,Cenit,hLo5Dg74PGHjSbcP6tNOr,Creí cuando dijeron que la luna se dormía cuan...,es,6hLo5Dg74PGHjSbcP6tNOr
3,Mexico City MX,Maskatesta,Tu Que Pediras,84Ztj2XGMvoWKILVJGasd,"Tee diiree una vez mass, lo quee pido por tii\...",es,284Ztj2XGMvoWKILVJGasd
4,Mexico City MX,DLD,Por Siempre,FLSb8CJwrcfXfzfD4ySji,Hoy he vuelto a ser quien soy\r\nla vida me re...,es,6FLSb8CJwrcfXfzfD4ySji
5,Mexico City MX,Los De Abajo,Skapate,mGNPivC5BpTowR3NF45bD,Uhh mami\r\nCon este ritmo va que me sabe a ch...,es,0mGNPivC5BpTowR3NF45bD
6,Mexico City MX,Los Caligaris,Kilómetros,Jr2cHZaIe1rmdVlc5efIy,Cada vez que pienso y me doy cuenta donde esto...,es,4Jr2cHZaIe1rmdVlc5efIy
7,Mexico City MX,Maria Daniela Y Su Sonido Lasser,Pobre Estúpida,Vu0aOYDeVPLe4dcVrvRvx,"Y ella de qué va, siempre está mirando atrás.\...",es,0Vu0aOYDeVPLe4dcVrvRvx
8,Mexico City MX,La Gusana Ciega,"Yes Sir, I Can Boogie",IwpFZxC7TlYBwVpXqks8n,Oye \r\nEstas perdiendo mucho tiempo \r\nNo du...,es,5IwpFZxC7TlYBwVpXqks8n
9,Mexico City MX,Liquits,Desde Que,QLfVqOrpBOJra53EhlEX0,"Te senti venir como en una vision, \r\nYa esta...",es,4QLfVqOrpBOJra53EhlEX0


In [160]:
lyrics.drop(columns=['song_id'], inplace=True)
lyrics.rename(columns={'id_fix': 'song_id'}, inplace=True)

In [162]:
lyrics.to_csv('./data/lyrics_with_lang.txt', index=False)

In [163]:
bad = pd.read_csv('./data/bad_ids/lyrics_with_lang_missing.txt')

In [164]:
print(len(bad))
print(len(lyrics))

24331
25325
